In [ ]:
!git clone --branch dev https://github.com/mikhailglukhoff/la_one_test.git
%cd la_one_test

In [ ]:
!pip install uv
!uv pip install -r requirements.txt

In [ ]:
!python main.py

In [2]:
from IPython.display import Markdown, display

with open("REPORT.md", "r", encoding="utf-8") as f:
    text = f.read()

display(Markdown(text))


# Отчёт по тестовому заданию
## Ход работы

1. Подготовка данных

* Загружен датасет Fashion Product Images (Small).
* Для обучения использованы как табличные признаки (категории товаров, цвет, сезон и пр., кроме описания товара), так и эмбеддинги изображений, извлечённые предобученной нейросетью.
* Для проверки гипотез смоделирована прокси-метрика rating, на основе топ-20% товаров с высоким рейтингом назначена метрика target=1. 
* В качестве товаров с высоким рейтингом выбраны подмножества категорий:
* masterCategory = {Apparel, Accessories}
* season = {Summer, Fall}
* usage = {Casual, Ethnic}
* gender = Women

2. Обучение моделей

* Табличная модель (CatBoost)
* Визуальная модель (эмбеддинги изображений)
* Комбинированная модель (табличные + визуальные признаки)
* Кластеризация эмбеддингов
* Использован k-means по пространству эмбеддингов изображений.
* Для каждого кластера посчитана доля успешных/неуспешных товаров.
### Результаты метрик

| Модель           | Accuracy | F1 (класс 1) | ROC-AUC |
|-----------------|---------|---------------|---------|
| Табличная       | 0.79    | 0.88          | 0.867   |
| Визуальная      | 0.77    | 0.86          | 0.820   |
| Комбинированная | 0.75    | 0.86          | 0.866   |


Табличная модель показывает наилучший ROC-AUC.

Визуальная модель слабее, но всё же хорошо улавливает сигнал от фото.

Комбинированная модель не дала значимого прироста по сравнению с табличной (ROC-AUC почти такой же).

### Важность признаков (топ)

Табличная модель: gender, masterCategory, season, usage.

Комбинированная модель: те же категории остаются доминирующими, но среди топ-10 появились и отдельные эмбеддинги (emb_1547, emb_1200, и др.).

Это значит, что модель действительно увидела категории с высоким рейтингом, они формируют основной сигнал.

### Кластеризация эмбеддингов

Лучшие по успеху кластеры:

* Кластер 4: 96.7% успешных
* Кластер 8: 88.8%

Худшие:

* Кластер 2: 41% успешных
* Кластер 6: 45%

Это указывает на то, что изображения содержат значимую информацию: в отдельных кластерах доля успешных товаров почти в 2 раза выше, чем в других.

### Выводы

Фото действительно влияют, но вклад категориальных признаков выше. 
Табличная модель без фото даёт лучший ROC-AUC.

Комбинированный подход не улучшает метрики по сравнению с табличной моделью, но даёт сигнал, что отдельные эмбеддинги могут быть полезными.

Кластеры эмбеддингов показывают сильный перекос в успешности. Это можно использовать для рекомендаций, проанализировав типы "успешных" изображений. 

Заданные категории (masterCategory, season, usage, gender) — действительно ключевые и определяющие.

В тесте были использованы изображения с низким разрешением.
При использовании качественных изображений вклад визуальной модели может значительно вырасти.
